<a id='startpage'></a>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение данных

In [3]:
import pandas as pd

from IPython.display import display

from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


### Описание данных
*	сalls — количество звонков,
*	minutes — суммарная длительность звонков в минутах,
*	messages — количество sms-сообщений,
*	mb_used — израсходованный интернет-трафик в Мб,
*	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Целевая переменная находится в столбце is_ultra, данные категориальные (бинарный целевой признак). Признаки - в остальных столбцах.

## Разбивка данных на выборки

Для более корректной работы моделей необходимо убрать лишние признаки, коррелирующие между собой.
У нас есть столбцы `calls` и `minutes`, которые взаимосвязаны. Для оценки силы этой взаимосвязи, посмотрим  на коэффициент корреляции.

In [5]:
df['calls'].corr(df['minutes'])

0.9820832355742293

Коэффициент корреляции по шкале Чеддока очень высокий. `calls` и `minutes` - коллинеарные признаки. Удалим `calls`.

In [6]:
df = df.drop(['calls'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   minutes   3214 non-null   float64
 1   messages  3214 non-null   float64
 2   mb_used   3214 non-null   float64
 3   is_ultra  3214 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 100.6 KB


### Обучающая, валидационная и тестовая выборки

Разделим наши данные в соотношении 60/20/20 - обучающая/валидационная/тестовая выборки.

In [7]:
df_train, df_valid = train_test_split(df, test_size=0.40, random_state=12345) # отделим 40% данных от обучающей выборки
df_valid, df_test = train_test_split(df_valid, test_size=0.50, random_state=12345) # разделим поровну данные для валидационной и тестовой выборки
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 4)
(643, 4)
(643, 4)


In [8]:
features_train = df_train.drop(['is_ultra'],axis = 1) # Переменные для обучающей выборки
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'],axis = 1) # Переменные для валидационной выборки
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'],axis = 1) # Переменные для тестовой выборки
target_test = df_test['is_ultra']

## Исследование моделей

Перед нами задача бинарной классификации. Исходя из этого исследуемыми моделями будут:
* "Дерево решений";
* "Случайный лес";
* "Метод логической регрессии".

### Модель "Дерево решений"

Создадим и обучим модель, выбрав произвольный параметр `max_depth`, равным 15.
Для постоянства результата зададим `random_state`, равный 12345.

In [9]:
model = DecisionTreeClassifier(random_state=12345, max_depth=15)
model.fit(features_train, target_train) # обучим модель
predictions = model.predict(features_valid) # получим предсказания модели
result = accuracy_score(target_valid, predictions).round(4) # посчитаем качество модели, округлим результат
print('Accuracy:', result)

Accuracy: 0.7449


Подберем наилучший гиперпараметр для модели, варьируя с `max_depth` в диапазоне (1, 60):

In [10]:
best_model = None
best_result = 0
for depth in range(1, 60):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth) # модель с заданной глубиной дерева
    model_dt.fit(features_train, target_train) # обучим модель
    predictions = model_dt.predict(features_valid) # получим предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели
    if result > best_result:
        best_model = model_dt
        best_result = result
        
print("Accuracy лучшей модели DecisionTree:", best_result.round(4), "\nЛучшая модель:", best_model)

Accuracy лучшей модели DecisionTree: 0.7885 
Лучшая модель: DecisionTreeClassifier(max_depth=4, random_state=12345)


### Модель "Случайный лес"

Создадим и обучим модель `RandomForest`, задав гиперпараметры по умолчанию.

In [11]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid).round(4)
print('Accuracy:', result)

Accuracy: 0.7792


Подберем гиперпараметры для модели: `n_estimators`, `max_depth`:

In [12]:
best_model = None
best_result = 0

for est in range(10, 61, 10):
    for depth in range(1, 11):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rf.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result = model_rf.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
        if result > best_result:
            best_model = model_rf # сохраним наилучшую модель
            best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели RandomForest:", best_result.round(4), "\nЛучшая модель:", best_model)

Accuracy наилучшей модели RandomForest: 0.7994 
Лучшая модель: RandomForestClassifier(max_depth=8, n_estimators=20, random_state=12345)


### Модель "Логическая регрессия"

Добавим дополнительные гиперпараметры: solver='lbfgs', который позволяет выбрать алгоритм, который будет строить модель и max_iter=1000 - максимальное количество итераций обучения.

In [13]:
model_lgr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_lgr.fit(features_train, target_train) # обучим модель на тренировочной выборке
result = model_lgr.score(features_valid, target_valid).round(4) # получим метрику качества модели на валидационной выборке

print("Accuracy модели LogisticRegression:", result, "\nМодель:", model_lgr)

Accuracy модели LogisticRegression: 0.7076 
Модель: LogisticRegression(max_iter=1000, random_state=12345)


### Вывод
Мы обучили "Дерево решений", "Случайный лес" и "Логическую регрессию". Самое высокое значение доли правильных ответов `accuracy` показала модель "Случайный лес" - 0.79005. Следовательно, эта модель (из рассмотренных нами) наилучшим образом подходит для предсказания поставленной задачи.

In [14]:
%%time
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
    'n_estimators': [50, 100]
}
# Create a based model
rf = RandomForestClassifier(random_state=12345)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, scoring="f1",
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(features_train, target_train)


# Return set of parameters with the best performance
print(grid_search.best_params_)
# Return the performance metric score
print(grid_search.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   0.3s
[CV] END boo

## Проверка модели на тестовой выборке

Лучше всего на валидационной выборке показала себя модель "Случайный лес". Проверим эту модель с подобранными нами гиперпараметрами на тестовой выборке.

In [14]:
model_rf = RandomForestClassifier(max_depth=8, n_estimators=20, random_state=12345)
model_rf.fit(features_train, target_train)
result_rf_test = model_rf.score(features_test, target_test).round(4) # посчитаем качество модели на тестовой выборке
print("Accuracy наилучшей модели RandomForest на тестовой выборке:", result_rf_test)

Accuracy наилучшей модели RandomForest на тестовой выборке: 0.8025


Протестируем остальные модели на нашей тестовой отложенной выборке.

In [15]:
predictions = model_dt.predict(features_test) # получим предсказания модели DecisionTree
result_dt_test = accuracy_score(target_test, predictions).round(4) # посчитаем качество модели на тестовой выборке
print("Accuracy лучшей модели DecisionTree на тестовой выборке:", result_dt_test)

result_lgr_test = model_lgr.score(features_test, target_test).round(4) # получим метрику качества модели на валидационной выборке
print("Accuracy модели LogisticRegression на тестовой выборке:", result_lgr_test)

Accuracy лучшей модели DecisionTree на тестовой выборке: 0.7123
Accuracy модели LogisticRegression на тестовой выборке: 0.6983


Создадим сводную таблицу `summary`, куда поместим результаты наших исследований.

In [16]:
summary = pd.DataFrame(data = [['DecisionTree', 0.7885, 0.7123], ['RandomForest', 0.7994, 0.8025],
                               ['LogisticRegression', 0.7076, 0.6983]], 
                     columns = ['model', 'accuracy модели', 'accuracy тестовой выборки'])
summary

,model,accuracy модели,accuracy тестовой выборки
0,DecisionTree,0.7885,0.7123
1,RandomForest,0.7994,0.8025
2,LogisticRegression,0.7076,0.6983


**Из рассмотренных нами моделей наилучшей для решения данной задачи оказалась модель "Случайный лес", `accuracy` на тестовой выборке которой - 0.8025.**

## (бонус) Проверка модели на адекватность

### Вариант 1
Самый частотный класс в целевом признаке - 0, то есть за константную модель берем такую, в которой все прогнозы 0. Чтобы оценить точность, разделим количество фактических нулей, на все объекты. Далее сравним полученную точность с `accuracy` RandomForest модели (0.7994).

In [17]:
df['is_ultra'].value_counts() / df.shape[0]

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Как мы видим, качество полученной модели выше константной, следовательно, наша модель адекватна.

### Вариант 2
Проверку на адекватность реализуем с помощью DummyClassifier, задав `strategy=most_frequent` (метод прогнозирования всегда возвращает наиболее часто встречающуюся метку класса в наблюдаемом аргументе).

In [18]:
model_dummy = DummyClassifier(strategy="most_frequent")
model_dummy.fit(features_train, target_train)
predictions = model_dummy.predict(features_valid) # получим предсказания модели
result_dummy = accuracy_score(target_valid, predictions).round(4) # посчитаем качество модели, округлим результат
print('Accuracy Dummy модели:', result_dummy)
print('Accuracy RandomForest модели: 0.7994')
if result_dummy < 0.7994:
    print('Модель RandomForest адекватна')
else:
    print('Модель RandomForest не адекватна')

Accuracy Dummy модели: 0.7061
Accuracy RandomForest модели: 0.7994
Модель RandomForest адекватна


Вернуться в [начало](#startpage).